In [ ]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation,BatchNormalization, Flatten, Conv2D, MaxPooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
# import keras.backend as K
# K.set_image_data_format(‘channels_last’)
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

In [ ]:
!wget -cq http://opensurfaces.cs.cornell.edu/static/minc/minc-2500.tar.gz

In [ ]:
import tarfile
tar = tarfile.open("minc-2500.tar.gz")
tar.extractall()
tar.close()

In [ ]:
original_images_dir = '/content/minc-2500/images'
sub_dirs = ['metal', 'glass', 'fabric', 'leather', 'paper', 'stone', 'wood', 'plastic', 'water', 'foliage']

In [ ]:
import os
train_dir = '/content/train'
val_dir = '/content/val'
for sub_dir in sub_dirs:
    os.makedirs(train_dir+"/"+sub_dir, exist_ok=True)
for sub_dir in sub_dirs:
    os.makedirs(val_dir+"/"+sub_dir, exist_ok=True)

In [ ]:
import cv2
for sub_dir in sub_dirs:
    cur_folder = original_images_dir+"/"+sub_dir
    for i,filename in enumerate(os.listdir(cur_folder)):
        img = cv2.imread(cur_folder+"/"+filename)
        if i>1999:
            cv2.imwrite(val_dir+"/"+sub_dir+"/"+filename, img)
        else:
            cv2.imwrite(train_dir+"/"+sub_dir+"/"+filename, img)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255 and image augmentation has also been used to reduce variance in our training
train_datagen = ImageDataGenerator(
      rescale=1./255,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      vertical_flip=True,
      rotation_range=20,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 140 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/content/train/',  # This is the source directory for training images
        target_size=(150, 150),
        batch_size=256,
        class_mode='categorical')

# Flow validation images in batches of 30 using validation_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        '/content/val/',  # This is the source directory for validation images
        target_size=(150, 150),  
        batch_size=256,
        class_mode='categorical')

Found 20000 images belonging to 10 classes.
Found 5000 images belonging to 10 classes.


In [ ]:
import os
#tf.keras.backend.clear_session()
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras.layers import Dropout

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D

In [ ]:
from keras.layers import Dropout

In [ ]:
def AlexNet(input_shape):
    
  X_input = Input(input_shape)
  x_dropout = Dropout(0.3)(X_input)
  X = Conv2D(96,(11,11),strides = 4,name="conv0")(x_dropout)
  X = BatchNormalization(axis = 3 , name = "bn0")(X)
  X = Activation('relu')(X)

  X = MaxPooling2D((3,3),strides = 2,name = 'max0')(X)

  X = Conv2D(256,(5,5),padding = 'same' , name = 'conv1')(X)
  X = BatchNormalization(axis = 3 ,name='bn1')(X)
  X = Activation('relu')(X)

  X = MaxPooling2D((3,3),strides = 2,name = 'max1')(X)

  X = Conv2D(384, (3,3) , padding = 'same' , name='conv2')(X)
  X = BatchNormalization(axis = 3, name = 'bn2')(X)
  X = Activation('relu')(X)
  
  X = Conv2D(384, (3,3) , padding = 'same' , name='conv3')(X)
  X = BatchNormalization(axis = 3, name = 'bn3')(X)
  X = Activation('relu')(X)

  X = Conv2D(256, (3,3) , padding = 'same' , name='conv4')(X)
  X = BatchNormalization(axis = 3, name = 'bn4')(X)
  X = Activation('relu')(X)

  X = MaxPooling2D((3,3),strides = 2,name = 'max2')(X)

  X = Flatten()(X)

  X = Dense(4096, activation = 'relu', name = "fc0")(X)

  X = Dense(4096, activation = 'relu', name = 'fc1')(X) 

  X = Dense(10 ,activation='softmax',name = 'fc2')(X)

  model = Model(inputs = X_input, outputs = X, name='AlexNet')
  return model

In [ ]:
alex = AlexNet(train_generator[0][0].shape[1:])
alex.summary()

Model: "AlexNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
dropout (Dropout)            (None, 150, 150, 3)       0         
_________________________________________________________________
conv0 (Conv2D)               (None, 35, 35, 96)        34944     
_________________________________________________________________
bn0 (BatchNormalization)     (None, 35, 35, 96)        384       
_________________________________________________________________
activation (Activation)      (None, 35, 35, 96)        0         
_________________________________________________________________
max0 (MaxPooling2D)          (None, 17, 17, 96)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 17, 17, 256)       6146

In [ ]:
import tensorflow as tf

In [ ]:
checkpoint_filepath = '/content/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

In [ ]:
alex.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics=['accuracy'])

In [ ]:
alex.fit(train_generator, epochs=100, steps_per_epoch=79, validation_data = validation_generator, verbose = 1, validation_steps=20, callbacks=[model_checkpoint_callback, early_stopping_callback])

Epoch 1/100
79/79 [==============================] - 207s 2s/step - loss: 3.6558 - accuracy: 0.1310 - val_loss: 4.8396 - val_accuracy: 0.1108
Epoch 2/100
79/79 [==============================] - 176s 2s/step - loss: 2.2720 - accuracy: 0.1421 - val_loss: 2.3504 - val_accuracy: 0.1578
Epoch 3/100
79/79 [==============================] - 177s 2s/step - loss: 2.2235 - accuracy: 0.1734 - val_loss: 2.3032 - val_accuracy: 0.1622
Epoch 4/100
79/79 [==============================] - 178s 2s/step - loss: 2.0854 - accuracy: 0.2294 - val_loss: 2.4208 - val_accuracy: 0.1736
Epoch 5/100
79/79 [==============================] - 178s 2s/step - loss: 2.0207 - accuracy: 0.2627 - val_loss: 2.3604 - val_accuracy: 0.2022
Epoch 6/100
79/79 [==============================] - 176s 2s/step - loss: 1.9775 - accuracy: 0.2844 - val_loss: 2.1996 - val_accuracy: 0.2230
Epoch 7/100
79/79 [==============================] - 178s 2s/step - loss: 1.9343 - accuracy: 0.3083 - val_loss: 1.9479 - val_accuracy: 0.2950
Epoch 